In [ ]:
import gc
import os
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import matplotlib.patches as patches
from sklearn.utils import resample
import torchvision.models as models
from torch.nn import functional as F
from sklearn.preprocessing import StandardScaler
from torchvision import datasets, models
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix
from torch.utils.data import DataLoader, Dataset

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
patch_size = {patch_size}

**LOADING PATIENTS' DATA** 

In [ ]:
# reading patients' data to obtain ground labels 

clinical_df = pd.read_csv('PATH-TO-CLINIDAL-DATA-CSV')
clinical_df = clinical_df[['Patient ID', 'Biomarker']]
clinical_df['Biomarker'] = clinical_df['Biomarker'].apply(lambda x: 1 if x == 'Positive' else 0)

clinical_df.head(3) 

**LOADING PATCH DATA**

In [ ]:
# reading patch data

patch_info = pd.read_csv("PATH-TO-PATCH-INFO-DATA-CSV")
patch_info.head(3) 

In [ ]:
df = pd.merge(patch_info, clinical_df, left_on = 'WSI No.', right_on = 'Patient ID', how = 'inner') 
df['Label'] = df['Biomarker'] 

df.head(3) 

In [ ]:
df = df[df['Annotation Id'] != -1]
df['Annotation Id'] = df['Annotation Id'] + 1 
df = df.sort_values(by=["WSI No.", "Annotation Id", "Y Pixel", "X Pixel"]) 

In [ ]:
print(len(df))

**LOADING PATCH FEATURES** 

In [ ]:
# loading feature embeddings for patches 

patch_features = torch.load('PATH-TO-PATCH-EMBEDDINGS-FILE(.pth)', map_location='cpu', weights_only = False) 

In [ ]:
print(len(patch_features)) 

**DATASET SPLIT** 

In [ ]:
# reading train, val & test raw patches information into dataframes 

train_ids = ["List of Train WSI Ids"]
val_ids = ["List of Val WSI Ids"]
test_ids = ["List of Test WSI Ids"]

train_df = df[df['WSI No.'].isin(train_ids)] 
train_feats = [patch_features[x] for x in train_df['Embedding No.'].to_list()] 

val_df = df[df['WSI No.'].isin(val_ids)] 
val_feats = [patch_features[x] for x in val_df['Embedding No.'].to_list()] 

test_df = df[df['WSI No.'].isin(test_ids)] 
test_feats = [patch_features[x] for x in test_df['Embedding No.'].to_list()] 

In [ ]:
train_wsi_nos = train_df['WSI No.'].to_list() 
train_ann_nos = train_df['Embedding No.'].to_list() 
train_labels_ = train_df['Label'].to_list() 

val_wsi_nos = val_df['WSI No.'].to_list() 
val_ann_nos = val_df['Embedding No.'].to_list() 
val_labels_ = val_df['Label'].to_list() 

test_wsi_nos = test_df['WSI No.'].to_list() 
test_ann_nos = test_df['Embedding No.'].to_list() 
test_labels_ = test_df['Label'].to_list() 

**AUTOENCODER FOR DIMENSIONALITY REDUCTION** 

In [ ]:
# Defining Autoencoder model

class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
           nn.Linear(input_size, 16),
           nn.ReLU(),
           nn.Linear(16, encoding_dim),
           nn.ReLU()
        )
        self.decoder = nn.Sequential(
           nn.Linear(encoding_dim, 16),
           nn.ReLU(),
           nn.Linear(16, input_size),
           nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train_feats) 

In [ ]:
# Setting random seed for reproducibility
torch.manual_seed(42)

input_size = 1000  # Number of input features
encoding_dim = 224  # Desired number of output dimensions
model = Autoencoder(input_size, encoding_dim)
#model = model.to(device) 

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr={LEARNING_RATE})

# Training the autoencoder
num_epochs = {NUMBER-OF-EPOCHS}
X_tensor = torch.FloatTensor(X_scaled) 
#X_tensor = X_tensor.to(device) 

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, X_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Loss for each epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
del X_scaled
del X_tensor
gc.collect()

In [ ]:
# Combining feature maps 

scaler = StandardScaler()

train_feats = torch.FloatTensor(scaler.fit_transform(train_feats))
val_feats = torch.FloatTensor(scaler.transform(val_feats))
test_feats = torch.FloatTensor(scaler.transform(test_feats))

In [ ]:
# Encoding the data using the trained autoencoder 

train_encoded_feats = model.encoder(train_feats).detach().cpu().numpy() 
del train_feats

val_encoded_feats = model.encoder(val_feats).detach().cpu().numpy() 
del val_feats

test_encoded_feats = model.encoder(test_feats).detach().cpu().numpy() 
del test_feats

gc.collect()

**BAG FORMATION** 

In [ ]:
# Function to create and return bags (1 bag / annotation) for each dataset 

def create_bags(wsis, anns, feature_maps, labels_, min_patches, max_patches): 
    
    bags = [] 
    labels = [] 
    
    wsis_against_bags = []  # list to maintain wsi numbers against bags 
    
    count = 0 
    extra_bags = 0 
    
    discarded_anns = 0 
    total_anns = 0 
    
    total_wsis = 1 
    discarded_wsis = 0 
    prev_wsi_len = 0 
    
    i = 0 
    reserve = []
    
    # loop to traverse through each WSI 
    while (i < len(wsis)): 
        
        curr_wsi = wsis[i] 
        bag = [] 
        curr_ann = anns[i] 
        
        # initial length of bag before traversing through annotation 
        prev_len = len(bags) 
        
        # loop to traverse through each annotation 
        while (i < len(anns) and curr_ann == anns[i] and curr_wsi == wsis[i]): 
            curr_ann = anns[i] 
            bag.append(feature_maps[i]) 
            i += 1 
        
        # minimum number of patches in a bag to keep it 
        if len(bag) >= min_patches: 
            # maximum number of patches in a bag before splitting it 
            if len(bag) < max_patches: 
                bags.append(bag) 
                labels.append(labels_[i - 1]) 
                wsis_against_bags.append(curr_wsi) 
            
            else: 
                x = 0 
                while (x + max_patches) < len(bag): 
                    bags.append(bag[x : x + max_patches]) 
                    labels.append(labels_[i - 1]) 
                    wsis_against_bags.append(curr_wsi) 
                    x += max_patches 
        
        # len(bag) < min patches 
        else: 
            if i < len(wsis) and wsis[i] == curr_wsi: 
                reserve.extend(bag) 
        
        # new length of bag after traversing through annotation 
        new_len = len(bags) 
        
        # new wsi 
        if i < len(wsis) and wsis[i] != curr_wsi: 

            extra_bags_size = int(min_patches * (3 / 2)) 
            
            x = 0 
            # loop to off load reserve once done with a given wsi (min_patches x 2 instances in each extra bag) 
            while (x + extra_bags_size) <= len(reserve): 
                extra_bags += 1 
                bags.append(reserve[x : x + extra_bags_size]) 
                labels.append(labels_[i - 1]) 
                wsis_against_bags.append(curr_wsi) 
                x += extra_bags_size 
                
            # refresh reserve for next wsi 
            reserve = [] 
            
            # if wsi discarded 
            total_wsis += 1 
            if len(bags) == prev_wsi_len: 
                discarded_wsis += 1 
            prev_wsi_len = len(bags) 
        
        # if annotation discarded 
        if prev_len == new_len: 
            discarded_anns += 1 
        total_anns += 1 
    
    # calculating percentage of discarded wsis for a given min_patches value 
    perc_wsis_discarded = discarded_wsis / total_wsis 
    
    return bags, extra_bags, perc_wsis_discarded, wsis_against_bags, labels

In [ ]:
# creating bags 

train_bags, _, _, train_wsis, train_labels = create_bags(train_wsi_nos, train_ann_nos, train_encoded_feats, train_labels_, 50, 224) 
val_bags, _, _, val_wsis, val_labels = create_bags(val_wsi_nos, val_ann_nos, val_encoded_feats, val_labels_, 50, 224) 
test_bags, _, _, test_wsis, test_labels = create_bags(test_wsi_nos, test_ann_nos, test_encoded_feats, test_labels_, 50, 224) 

In [ ]:
print(len(train_bags), len(train_wsis), len(train_labels)) 
print(len(val_bags), len(val_wsis), len(val_labels)) 
print(len(test_bags), len(test_wsis), len(test_labels)) 

**CLASS DISTRIBUTION** 

In [ ]:
# class distribution of train, validation, and test datasets 

def plot_bag_labels(ax, title, labels):
    ax.bar(['Biomarker (-)', 'Biomarker (+)'], [labels.count(0), labels.count(1)])
    ax.set_title(title)
    ax.set_ylabel('Count')

fig, axs = plt.subplots(1, 3, figsize=(10, 4))

plot_bag_labels(axs[0], 'TRAIN BAG LABELS', train_labels)
plot_bag_labels(axs[1], 'VALIDATION BAG LABELS', val_labels)
plot_bag_labels(axs[2], 'TEST BAG LABELS', test_labels)

plt.tight_layout()
plt.show()

**CUSTOM DATASETS & DATA LOADERS** 

In [ ]:
# Custom dataset to load feature vectors & binary class labels 

class CustomDataset(Dataset):
    def __init__(self, bags, labels):
        self.bags = bags
        self.labels = labels

    def __len__(self):
        return len(self.bags)

    def __getitem__(self, idx):
        
        x = self.bags[idx]
        if len(x) > 224:
            feat_map = x[:224]
        else:
            feat_map = []
            feat_map.extend([np.array([0.0] * 224)] * int((224 - len(x)) / 2)) # black padding above
            feat_map.extend(x) # feature map of patches in between
            feat_map.extend([np.array([0.0] * 224)] * int(((224 - len(x)) / 2) + 1)) # black padding below
            feat_map = feat_map[:224]
        
        x = np.array(feat_map) 
        
        x = torch.tensor(x).float()
        x = nn.functional.normalize(x, dim=0, p=2)  # p=2 for L2 norm, dim=0 for cols
        bag = x
        label = self.labels[idx]

        return torch.stack([bag]), label

In [ ]:
# Create instances of custom datasets

train_dataset = CustomDataset(train_bags, train_labels)
valid_dataset = CustomDataset(val_bags, val_labels)
test_dataset = CustomDataset(test_bags, test_labels) 

In [ ]:
batch_size = {BATCH-SIZE}
num_of_workers = {NUMBER-OF-WORKERS}

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = num_of_workers)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers = num_of_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = num_of_workers)

**FINAL MODEL** 

In [ ]:
# defining final pipeline to process prepared bags 

class Pipeline(nn.Module):
    def __init__(self,num_classes):
        #define necessary layers
        super().__init__()
        self.num_classes = num_classes
          
        self.base = models.{'MODEL_NAME'}(weights=True)
        
        # Unfreeze model weights
        for param in self.base.parameters():
            param.requires_grad = True
        
        self.flatten = nn.Flatten()
        
        self.head = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ) 
    
    def forward(self,X):
        X = self.base(X)
        X = self.flatten(X)
        X = self.head(X)
        return X, F.sigmoid(X)

pos_weight = torch.tensor([POSITIVE-WEIGHT])
pos_weight = pos_weight.to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
model = Pipeline(1)

In [ ]:
# modifying first layer to input color channel 
# for efficientnetb0
model.base.features[0][0] = nn.Conv2d(1, 32,kernel_size= (3,3), stride = 2, padding= 1, bias=False)

# for resnet50
#model.base.conv1=nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# for VIT
#model.base.conv_proj = nn.Conv2d(1, 768, kernel_size=(32, 32), stride=(32, 32))

# Similarly can be done for other models

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    model = model.to('cuda')
    print('available')

optimizer = optim.Adam(params=model.parameters(), lr={LEARNING-RATE}) 

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
# Initialize empty lists to store loss and accuracy for training and validation 

train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []
roc_values_train = []
roc_values_val = []

In [ ]:
# Training loop
num_epochs = {NUMBER-OF-EPOCHS}

for epoch in range(num_epochs):
    model.train()
    y_true_train = []
    y_scores_train = []
    train_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs_without_sigmoid, outputs_with_sigmoid = model(images)
        loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()  
        predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true_train.extend(labels.cpu().detach().numpy())
        y_scores_train.extend(outputs_with_sigmoid.cpu().detach().numpy())

    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(100 * correct / total)

    fpr_train, tpr_train, _ = roc_curve(y_true_train, y_scores_train)
    roc_auc_train = auc(fpr_train, tpr_train)
    roc_values_train.append(roc_auc_train)

    # Plot ROC curve for the training set

    # Validate your model after each epoch
    model.eval()
    valid_loss = 0
    correct = 0
    total = 0
    y_true_val = []
    y_scores_val = []
    with torch.no_grad():
        for images, labels in tqdm(valid_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs_without_sigmoid, outputs_with_sigmoid = model(images)
            loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
            valid_loss += loss.item()  
            predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            y_true_val.extend(labels.cpu().detach().numpy())
            y_scores_val.extend(outputs_with_sigmoid.cpu().detach().numpy())

    valid_losses.append(valid_loss / len(valid_loader))
    valid_accuracies.append(100 * correct / total)

    fpr_val, tpr_val, _ = roc_curve(y_true_val, y_scores_val)
    roc_auc_val = auc(fpr_val, tpr_val)
    roc_values_val.append(roc_auc_val)

    # Classification report
    print(f'Epoch {epoch + 1}, Train Accuracy: {train_accuracies[-1]:.2f}%, Train Loss: {train_losses[-1]:.2f}%, Train ROC-AUC: {roc_auc_train:.2f}%, Val Accuracy: {valid_accuracies[-1]:.2f}%, Val Loss: {valid_losses[-1]:.2f}%, Val ROC-AUC: {roc_auc_val:.2f}%')

    y_true_val = np.array(y_true_val)
    y_pred_val = (np.array(y_scores_val) >= 0.5).astype(int)
    print("Validation Classification Report:")
    print(classification_report(y_true_val, y_pred_val))

In [ ]:
def display_training_curves(train_losses, valid_losses, train_accuracies, valid_accuracies, subplot):
    if subplot % 10 == 1:  
        plt.subplots(figsize=(12, 5), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(train_losses)
    ax.plot(valid_losses)
    ax.set_title('Model Loss')
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    ax.legend(['Train', 'Valid'])

    ax = plt.subplot(subplot + 1)
    ax.set_facecolor('#F8F8F8')
    ax.plot(train_accuracies)
    ax.plot(valid_accuracies)
    ax.set_title('Model Accuracy')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Epoch')
    ax.legend(['Train', 'Valid'])

In [ ]:
# Plot the training curves
display_training_curves(train_losses, valid_losses, train_accuracies, valid_accuracies, 121)

**EVALUATING** 

In [ ]:
model.eval()
test_loss = 0
correct = 0
total = 0
y_true_test = []
y_pred_test = []
y_probs = []

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs_without_sigmoid, outputs_with_sigmoid = model(images)
        loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
        test_loss += loss.item() 
        predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        y_true_test.extend(labels.cpu().detach().numpy())
        y_pred_test.extend(predicted.cpu().detach().numpy())
        y_probs.extend(outputs_with_sigmoid.cpu().detach().numpy())

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

# Calculate ROC-AUC for test set
fpr_test, tpr_test, _ = roc_curve(y_true_test, y_probs)
roc_auc_test = auc(fpr_test, tpr_test)

print(f'Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.2f}%')

# Classification report for test set
print("Test Classification Report:")
print(classification_report(y_true_test, y_pred_test))

In [ ]:
# ROC curve for the test set
plt.figure(figsize=(8, 8))
plt.plot(fpr_test, tpr_test, label=f'Test ROC Curve (AUC = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Test Set')
plt.legend()
plt.show()

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_true_test, y_pred_test)

# Plot confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Bag Level Confusion Matrix')
plt.show()

**AGGREGATING BAG LEVEL RESULTS TO WSI LEVEL** 

In [ ]:
# Obtaining WSI level label prediction using majority vote

wsi_true = {}
wsi_pred_counts = {}

# Group predictions by WSI and store true labels
for wsi, true_label, pred_label in zip(test_wsis, y_true_test, y_pred_test):
    if wsi not in wsi_true:
        wsi_true[wsi] = true_label
    
    if wsi not in wsi_pred_counts:
        wsi_pred_counts[wsi] = {0: 0, 1: 0}
    
    wsi_pred_counts[wsi][pred_label] += 1
    
wsi_true_labels = []
wsi_pred_labels = []

for wsi, counts in wsi_pred_counts.items():
    wsi_true_labels.append(wsi_true[wsi])
    
    # Choose the label with the maximum count (majority vote)
    majority_vote = 0 if counts[0] > counts[1] else 1
    wsi_pred_labels.append(majority_vote)

In [ ]:
wsi_pred_counts 

In [ ]:
# Classification report for test set 

print("Test WSI Level Classification Report:\n")
print(classification_report(wsi_true_labels, wsi_pred_labels)) 

In [ ]:
# calculating CI 

accuracies = []
for i in range(1000):
    y_true_sample, y_pred_sample = resample(wsi_true_labels, wsi_pred_labels)
    accuracies.append(accuracy_score(y_true_sample, y_pred_sample))

# Compute confidence interval
alpha = 0.95
lower = np.percentile(accuracies, (1 - alpha) / 2 * 100)
upper = np.percentile(accuracies, (1 + alpha) / 2 * 100)

print(f"Accuracy: {np.mean(accuracies):.3f} with 95% CI: [{lower:.3f}, {upper:.3f}]") 

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(wsi_true_labels, wsi_pred_labels)

# Plot confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('WSI Level Confusion Matrix')
plt.show()